In [5]:
import pandas as pd
import os
cwd = os.getcwd()

ROOT = os.path.join(cwd, '../')
DATA_FOLDER = os.path.join(ROOT, 'data/')

df = pd.read_pickle(os.path.join(DATA_FOLDER, 'saeb_treated_3-NO.pkl'))

df = df.dropna()
df.head()

,ano,sigla_uf,escola_publica,id_aluno,situacao_censo,disciplina,proficiencia,erro_padrao,proficiencia_saeb,erro_padrao_saeb,...,frequenta_festas_comunidade,gosta_estudar_disciplina,faz_licao_casa,correcao_licao_casa,utiliza_biblioteca_escola,rede,id_escola,serie,turno,desempenho_aluno_cat
27111,2015,PB,1,17523018,0,MT,-0.627672,0.648351,214.882332,36.237847,...,1.0,0.0,1.0,1.0,1.0,2,25083490,9,3,0
27112,2015,PB,1,17491737,0,LP,-1.027257,0.354996,193.389947,19.557923,...,2.0,1.0,2.0,2.0,2.0,2,61134907,9,1,0
27115,2015,PB,1,17491737,0,MT,-0.807123,0.639382,204.852398,35.736548,...,2.0,1.0,2.0,2.0,2.0,2,61134907,9,1,0
27122,2015,PB,1,17488937,0,LP,-0.341425,0.380753,231.174739,20.976963,...,1.0,1.0,2.0,2.0,1.0,2,25027573,9,1,1
27123,2015,PB,1,17488937,0,MT,-0.906444,0.545208,199.301116,30.472944,...,1.0,1.0,2.0,2.0,1.0,2,25027573,9,1,0


In [17]:
# distribuição das classes
df['desempenho_aluno'].value_counts(dropna=False)

desempenho_aluno
Básico          27142
Insuficiente    12943
Proficiente      9712
Name: count, dtype: int64

In [18]:
exclude = ['ano', 'sigla_uf', 'id_aluno', 'situacao_censo', 'disciplina', 'proficiencia', 'erro_padrao', 'proficiencia_saeb',
       'erro_padrao_saeb', 'sexo', 'raca_cor','ano_nascimento', 'rede', 'id_escola', 'serie', 'turno', 'escola_publica', 'desempenho_aluno']

In [19]:
# para cada coluna tirando as exclude, mostrar cada valor único
for col in df.drop(columns=exclude).columns:
    print(col, df[col].unique())

idade [18 17 16 15 14 13]
escolaridade_mae [4. 1. 0. 2. 3. 5.]
mae_sabe_ler_escrever [1 0]
mae_le [1. 0.]
escolaridade_pai [1. 3. 4. 2. 0. 5.]
pai_sabe_ler_escrever [1. 0.]
pai_le [1. 0.]
responsaveis_incentivam_realizacao_licao_casa [1. 0.]
responsaveis_incentivam_estudos [1. 0.]
responsaveis_incentivam_leitura [1. 0.]
responsaveis_incentivam_comparecer_aulas [1. 0.]
possui_computador [1. 0. 2. 3. 4.]
tempo_lazer [1. 4. 2. 3. 0.]
leitura_jornais [1. 0. 2.]
leitura_livros_geral [1. 2. 0.]
leitura_literatura_infantojuvenil [1. 2. 0.]
leitura_historia_quadrinhos [1. 2. 0.]
leitura_revista_comportamento [1. 2. 0.]
leitura_revistas_geral [1. 2. 0.]
leitura_internet [0. 1. 2.]
frequenta_biblioteca [1. 0. 2.]
frequenta_cinema [0. 2. 1.]
frequenta_espetaculo_exposicao [1. 2. 0.]
frequenta_festas_comunidade [1. 2. 0.]
gosta_estudar_disciplina [0. 1.]
faz_licao_casa [1. 2. 0.]
correcao_licao_casa [1. 2. 0.]
utiliza_biblioteca_escola [1. 2. 0.]
desempenho_aluno_cat [0 1 2]


In [20]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder

from IPython.display import clear_output

clear_output()

In [24]:
df['ano'].unique()

<IntegerArray>
[2015, 2017]
Length: 2, dtype: Int64

In [25]:
# select all columns with 3 unique valuea
columns_3 = [col for col in df.columns if len(df[col].unique()) == 3]
print(columns_3)

['desempenho_aluno', 'leitura_jornais', 'leitura_livros_geral', 'leitura_literatura_infantojuvenil', 'leitura_historia_quadrinhos', 'leitura_revista_comportamento', 'leitura_revistas_geral', 'leitura_internet', 'frequenta_biblioteca', 'frequenta_cinema', 'frequenta_espetaculo_exposicao', 'frequenta_festas_comunidade', 'faz_licao_casa', 'correcao_licao_casa', 'utiliza_biblioteca_escola', 'rede', 'turno', 'desempenho_aluno_cat']


In [34]:
X = df.drop(columns=exclude)[[col for col in columns_3 if col not in exclude and col != 'desempenho_aluno_cat']]
y = df['desempenho_aluno_cat']

In [35]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

y_train = to_categorical(y_train, num_classes=3)
y_test = to_categorical(y_test, num_classes=3)

In [36]:
# print unique values for each col
for col in range(X_train.shape[1]):
    print(col, np.unique(X_train[:, col]))

0 [0.  0.5 1. ]
1 [0.  0.5 1. ]
2 [0.  0.5 1. ]
3 [0.  0.5 1. ]
4 [0.  0.5 1. ]
5 [0.  0.5 1. ]
6 [0.  0.5 1. ]
7 [0.  0.5 1. ]
8 [0.  0.5 1. ]
9 [0.  0.5 1. ]
10 [0.  0.5 1. ]
11 [0.  0.5 1. ]
12 [0.  0.5 1. ]
13 [0.  0.5 1. ]


In [37]:
model = Sequential()
model.add(Dense(12, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(3, activation='softmax'))

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)

# Compilar o modelo
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

/home/erlo/.local/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_2 (Dense)                 │ (None, 12)             │           180 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 3)              │            39 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 219 (876.00 B)

 Trainable params: 219 (876.00 B)

 Non-trainable params: 0 (0.00 B)

In [38]:
history = model.fit(X_train, y_train, epochs=100, batch_size=32, callbacks=[es])

Epoch 1/100
1245/1245 ━━━━━━━━━━━━━━━━━━━━ 1s 554us/step - accuracy: 0.5436 - loss: 1.0012
Epoch 2/100
 301/1245 ━━━━━━━━━━━━━━━━━━━━ 0s 502us/step - accuracy: 0.5390 - loss: 0.9841

/home/erlo/.local/lib/python3.12/site-packages/keras/src/callbacks/early_stopping.py:155: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: accuracy,loss
  current = self.get_monitor_value(logs)


1245/1245 ━━━━━━━━━━━━━━━━━━━━ 1s 632us/step - accuracy: 0.5443 - loss: 0.9783
Epoch 3/100
1245/1245 ━━━━━━━━━━━━━━━━━━━━ 1s 502us/step - accuracy: 0.5474 - loss: 0.9662
Epoch 4/100
1245/1245 ━━━━━━━━━━━━━━━━━━━━ 1s 547us/step - accuracy: 0.5508 - loss: 0.9587
Epoch 5/100
1245/1245 ━━━━━━━━━━━━━━━━━━━━ 1s 566us/step - accuracy: 0.5488 - loss: 0.9612
Epoch 6/100
1245/1245 ━━━━━━━━━━━━━━━━━━━━ 1s 509us/step - accuracy: 0.5434 - loss: 0.9627
Epoch 7/100
1245/1245 ━━━━━━━━━━━━━━━━━━━━ 1s 543us/step - accuracy: 0.5463 - loss: 0.9594
Epoch 8/100
1245/1245 ━━━━━━━━━━━━━━━━━━━━ 1s 488us/step - accuracy: 0.5412 - loss: 0.9644
Epoch 9/100
1245/1245 ━━━━━━━━━━━━━━━━━━━━ 1s 518us/step - accuracy: 0.5480 - loss: 0.9569
Epoch 10/100
1245/1245 ━━━━━━━━━━━━━━━━━━━━ 1s 570us/step - accuracy: 0.5515 - loss: 0.9557
Epoch 11/100
1245/1245 ━━━━━━━━━━━━━━━━━━━━ 1s 517us/step - accuracy: 0.5478 - loss: 0.9568
Epoch 12/100
1245/1245 ━━━━━━━━━━━━━━━━━━━━ 1s 579us/step - accuracy: 0.5506 - loss: 0.9545
Epoch 13

In [43]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix

# Suponha que X seja o conjunto de features binárias e y as classes
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Instancia o modelo Random Forest
rf = RandomForestClassifier()

# Ajuste de hiperparâmetros (opcional)
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [5, 10, None],
    'min_samples_split': [2, 5, 10],
}

grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=3, verbose=2, n_jobs=2)
grid_search.fit(X_train, y_train)

# Melhor modelo
best_rf = grid_search.best_estimator_

# Previsão
y_pred = best_rf.predict(X_test)

# Avaliação
print("Matriz de Confusão:")
print(confusion_matrix(y_test, y_pred))

print("\nRelatório de Classificação:")
print(classification_report(y_test, y_pred))


Fitting 3 folds for each of 18 candidates, totalling 54 fits


/home/erlo/.local/lib/python3.12/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
/home/erlo/.local/lib/python3.12/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


[CV] END .max_depth=5, min_samples_split=2, n_estimators=100; total time=   0.7s
[CV] END .max_depth=5, min_samples_split=2, n_estimators=100; total time=   0.7s
[CV] END .max_depth=5, min_samples_split=2, n_estimators=100; total time=   0.6s
[CV] END .max_depth=5, min_samples_split=2, n_estimators=200; total time=   1.2s
[CV] END .max_depth=5, min_samples_split=2, n_estimators=200; total time=   1.2s
[CV] END .max_depth=5, min_samples_split=5, n_estimators=100; total time=   0.6s
[CV] END .max_depth=5, min_samples_split=2, n_estimators=200; total time=   1.2s
[CV] END .max_depth=5, min_samples_split=5, n_estimators=100; total time=   0.6s
[CV] END .max_depth=5, min_samples_split=5, n_estimators=100; total time=   0.6s
[CV] END .max_depth=5, min_samples_split=5, n_estimators=200; total time=   1.2s
[CV] END .max_depth=5, min_samples_split=5, n_estimators=200; total time=   1.2s
[CV] END max_depth=5, min_samples_split=10, n_estimators=100; total time=   0.6s
[CV] END .max_depth=5, min_s

In [60]:
# Dividir os dados em conjuntos de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# validação
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [61]:
print(y_train.shape)
print(X_train.shape)


(32227,)
(32227, 27)


In [62]:
# Construir o modelo
model = Sequential()
model.add(Dense(32, input_dim=X_train.shape[1], activation='relu', kernel_regularizer='l2'))
#model.add(Dropout(0.2))
model.add(Dense(2, activation='relu'))
model.add(Dense(len(set(y_train)), activation='softmax'))

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)

# Compilar o modelo
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

/home/erlo/.local/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_9 (Dense)                 │ (None, 32)             │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 2)              │            66 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 4)              │            12 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 974 (3.80 KB)

 Trainable params: 974 (3.80 KB)

 Non-trainable params: 0 (0.00 B)

In [63]:
# Treinar o modelo
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_val, y_val), callbacks=[es])

Epoch 1/50


ValueError: Arguments `target` and `output` must have the same shape. Received: target.shape=(None, 1), output.shape=(None, 4)